In [2]:
from platform import python_version
import sys
import pandas as pd
import numpy as np
import jqdatasdk as jq
import jqfactor_analyzer as ja
import numba
import yfinance as yf
print(sys.executable)
print(python_version())

/opt/homebrew/anaconda3/bin/python
3.11.3


In [3]:
jq.auth('15340861099','Shaojintian316!')
jq.get_account_info()

TTransportException: TTransportException(type=1, message="Could not connect to ('39.107.190.114', 7000)")

In [4]:
# 获取5日平均换手率因子2018-01-01到2018-12-31之间的数据（示例用从库中直接调取）
# 聚宽因子库数据获取方法在下方
from jqfactor_analyzer.sample import VOL5
factor_data = VOL5

# 对因子进行分析
far = ja.analyze_factor(
    factor_data,  # factor_data 为因子值的 pandas.DataFrame
    quantiles=10,
    periods=(1, 10),
    industry='jq_l1',
    weight_method='avg',
    max_loss=0.1
)

# 获取整理后的因子的IC值
far.ic

auth success 


Exception: 您的账号权限仅能获取2022-04-25至2023-05-02的数据，请调整时间参数后重试。如需更长时间的数据范围，可联系运营咨询采购

In [30]:
tqqq = yf.Ticker("TQQQ")

# get all stock info
tqqq.history(period="1mo")
#tqqq.history?

pandas.core.frame.DataFrame

In [30]:
#hushen 300

pandas.core.frame.DataFrame

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
